In [1]:
import plotly.graph_objects as go
import plotly.offline as py
from plotly.subplots import make_subplots
import plotly.io as pio

import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt
%matplotlib qt5

from gudhi.point_cloud import timedelay
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from ripser import ripser
from persim import plot_diagrams

import time

from praatio import textgrid

import soundfile as sf

# head_tail_scissor is to erase signal in head and tail that has amplitude smaller than 0.05
# can also use it to see if the length of renewing signal is greater than 500 or not 
def head_tail_scissor(sig):
    valid_interval=[index for index in range(len(sig)) if (sig[index]>0.03)]
    if len(valid_interval)==0:
        return False,sig
    head=min(valid_interval)
    tail=max(valid_interval)
    sig=sig[head:tail+1]
    if tail-head<500:
        return False,sig
    return True,sig

# principle_frequency_finder is to find the principle frequency of a speech signal
def principle_frequency_finder(sig):
    t=int(len(sig)/2)
    corr=np.zeros(t)

    for index in np.arange(t):
        ACF_delay=sig[index:]
        L=(t-index)/2
        m = np.sum(sig[int(t-L):int(t+L+1)]**2) + np.sum(ACF_delay[int(t-L):int(t+L+1)]**2)
        r = np.sum(sig[int(t-L):int(t+L+1)]*ACF_delay[int(t-L):int(t+L+1)])
        corr[index] = 2*r/m

    zc = np.zeros(corr.size-1)
    zc[(corr[0:-1] < 0)*(corr[1::] > 0)] = 1
    zc[(corr[0:-1] > 0)*(corr[1::] < 0)] = -1

    admiss = np.zeros(corr.size)
    admiss[0:-1] = zc
    for i in range(1, corr.size):
        if admiss[i] == 0:
            admiss[i] = admiss[i-1]

    maxes = np.zeros(corr.size)
    maxes[1:-1] = (np.sign(corr[1:-1] - corr[0:-2])==1)*(np.sign(corr[1:-1] - corr[2::])==1)
    maxidx = np.arange(corr.size)
    maxidx = maxidx[maxes == 1]
    max_index = 0
    if len(corr[maxidx]) > 0:
        max_index = maxidx[np.argmax(corr[maxidx])]

    return (max_index, corr)

In [ ]:
inputFile="E:\\MFA\\output\\ALL_049_F_ENG_ENG_HT1.Textgrid"
tg=textgrid.openTextgrid(inputFile,includeEmptyIntervals=False)
phoneTier=tg.tierDict['phones']
specified_vowel=['ŋ']
wavFile='E:\\MFA\\input\\ALL_049_F_ENG_ENG_HT1.wav'
sig,samplerate=sf.read(wavFile)
specified_vowel_list=[ele for ele in phoneTier.entryList if ele[2] in specified_vowel]
def wav_fraction_finder(start_time, end_time):
    sig_fraction=sig[int(start_time*samplerate):int(end_time*samplerate)]
    return sig_fraction

specified_valid_vowel_list=[head_tail_scissor(wav_fraction_finder(ele[0],ele[1]))[1] for ele in specified_vowel_list if head_tail_scissor(wav_fraction_finder(ele[0],ele[1]))[0]]

element=4
data=data=specified_valid_vowel_list[element]
AutoT,corr=principle_frequency_finder(np.array(specified_valid_vowel_list[element]))
AutoT=AutoT/samplerate

list=np.arange(10,1290,10)
AutoDelay=np.zeros((1,list.size),int)[0]
MP=np.zeros((1,list.size))[0]
BD=np.zeros((1,list.size))[0]
for M in list:
    index=np.where(list==M)[0][0]
    AutoDelay[index]=round(AutoT*samplerate*2*np.pi/M)
    point_Cloud=timedelay.TimeDelayEmbedding(M, AutoDelay[index],5)
    Points=point_Cloud(data)
    dgms = ripser(Points,maxdim=1)['dgms']
    dgms=dgms[1]
    persistent_time=[ele[1]-ele[0] for ele in dgms]            
    MaxEle=argmax(persistent_time)
    BD[index]=dgms[MaxEle][0]
    MP[index]=persistent_time[MaxEle]


In [ ]:
M_scatter=[10, 100, 1000, 1280]
compute_time=[0, 0, 0, 0]
delay=[round(AutoT*samplerate*2*np.pi/ele) for ele in M_scatter]
X_list=[]
Y_list=[]
Z_list=[]
for element in range(4):
    point_Cloud=timedelay.TimeDelayEmbedding(M_scatter[element], delay[element], 1)
    Points=point_Cloud(data)
    Trans=StandardScaler().fit_transform(Points)
    pca=PCA(n_components=3,whiten=True)
    X_PCA=pca.fit_transform(Trans)
    PCA_len=len(X_PCA)
    X=np.zeros((1,PCA_len))
    Y=np.zeros((1,PCA_len))
    Z=np.zeros((1,PCA_len))
    X=X_PCA[:,0]
    X_list.append(X)
    Y=X_PCA[:,1]
    Y_list.append(Y)
    Z=X_PCA[:,2]
    Z_list.append(Z)

    t=time.time()
    dgms = ripser(Points,maxdim=1)['dgms']
    compute_time[element]=time.time()-t

print(compute_time)

In [9]:
fig = make_subplots(
    rows=2, cols=4,
    column_widths=[0.25, 0.25, 0.25, 0.25],
    row_heights=[0.5, 0.5],
    vertical_spacing=0.05,
    horizontal_spacing=0.025,
    specs=[[{"type": "scatter", "colspan": 2}, None, {"type": "scatter3d"},{"type": "scatter3d"}],
           [ {"type": "scatter"}, {"type": "scatter"},{"type": "scatter3d"},{"type": "scatter3d"}]])

fig.add_trace(
    go.Scatter(
    x=np.array(range(len(data))), y=data,
    marker=dict(
        size=2,
        colorscale='Viridis',
    ),
    line=dict(
        color='#000000',
        width=1
    ),
),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
    x=np.array(range(MP.size))*10+10, y=MP,
    marker=dict(
        size=2,
        colorscale='Viridis',
    ),
    line=dict(
        color='#000000',
        width=1
    )
),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
    x=np.array(range(AutoDelay.size))*10+10, y=AutoDelay,
    marker=dict(
        size=2,
        colorscale='Viridis',
    ),
    line=dict(
        color='#000000',
        width=1
    )
),
    row=2, col=2
)

for ele in range(4):
    fig.add_trace(
        go.Scatter3d(
        x=X_list[ele], y=Y_list[ele], z=Z_list[ele],
        marker=dict(
            size=3,
            color=Z_list[ele],
            colorscale='Viridis',#'sunset','thermal',
        ),
        line=dict(
            color='#0000b3',
            width=1
        )
    ),
        row=int(ele/2)+1, col=ele%2+3
    )

fig.update_layout(
    margin=dict(r=1, t=20, b=1, l=1),
    showlegend=False,
    template='simple_white',
)

fig.update_layout(
    scene = dict(
        xaxis = dict(tickfont=dict(size=10),tickvals= [-1,0]),
        xaxis_title="",
        yaxis = dict(tickfont=dict(size=10),tickvals= [-1,0]),
        yaxis_title="",
        zaxis = dict(tickfont=dict(size=10),nticks=4),
        zaxis_title="",
        camera=dict(
            up=dict(
                x=0.3,
                y=0.3,
                z=1
            ),
            eye=dict(
                x=1.25,
                y=1.25,
                z=1.25,
            )
        ),),
    scene3 = dict(
        xaxis = dict(tickfont=dict(size=10),tickvals= [-1,0]),
        xaxis_title="",
        yaxis = dict(tickfont=dict(size=10),tickvals= [-1,0]),
        yaxis_title="",
        zaxis = dict(tickfont=dict(size=10),nticks=4),
        zaxis_title="",
        camera=dict(
            up=dict(
                x=0.3,
                y=0.3,
                z=1
            ),
            eye=dict(
                x=1.25,
                y=1.25,
                z=1.25,
            )
        ),),
    scene4 = dict(
        xaxis = dict(tickfont=dict(size=10),tickvals= [-1,0]),
        xaxis_title="",
        yaxis = dict(tickfont=dict(size=10),tickvals= [-1,0]),
        yaxis_title="",
        zaxis = dict(tickfont=dict(size=10),nticks=4),
        zaxis_title="",
        camera=dict(
            up=dict(
                x=0.3,
                y=0.3,
                z=1
            ),
            eye=dict(
                x=1.25,
                y=1.25,
                z=1.25,
            )
        ),),

    scene2 = dict(
    xaxis = dict(tickfont=dict(size=10),tickvals= [-1,0]),
    xaxis_title="x",
    yaxis = dict(tickfont=dict(size=10),tickvals= [-1,0]),
    yaxis_title="y",
    zaxis = dict(tickfont=dict(size=10),nticks=4),
    zaxis_title="z",
    camera=dict(
        up=dict(
            x=0,
            y=0,
            z=1,
        ),
        eye=dict(
            x=2.,
            y=-.5,
            z=.5,
        )
    ),),
)

fig.show()
#pio.write_image(fig, 'E:/Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/discussion_dimRefine3.pdf',scale=1, width=1080, height=500)
